<a href="https://colab.research.google.com/github/eisenjulian/clarify/blob/main/followup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! wget https://storage.googleapis.com/bleurt-oss-21/BLEURT-20-D6.zip
! unzip BLEURT-20-D6.zip

--2024-11-17 11:10:28--  https://storage.googleapis.com/bleurt-oss-21/BLEURT-20-D6.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.181.207, 142.251.183.207, 173.194.206.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.181.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 171135344 (163M) [application/octet-stream]
Saving to: ‘BLEURT-20-D6.zip’

BLEURT-20-D6.zip    100%[===================>] 163.21M   126MB/s    in 1.3s    

2024-11-17 11:10:29 (126 MB/s) - ‘BLEURT-20-D6.zip’ saved [171135344/171135344]

Archive:  BLEURT-20-D6.zip
   creating: BLEURT-20-D6/
  inflating: BLEURT-20-D6/bert_config.json  
  inflating: BLEURT-20-D6/saved_model.pb  
   creating: BLEURT-20-D6/variables/
  inflating: BLEURT-20-D6/variables/variables.index  
  inflating: BLEURT-20-D6/variables/variables.data-00000-of-00001  
  inflating: BLEURT-20-D6/sent_piece.vocab  
  inflating: BLEURT-20-D6/bleurt_config.json  
  inflating: BLEU

In [ ]:
! pip install datasets openai sacrebleu https://github.com/google-research/bleurt/archive/b610120347ef22b494b6d69b4316e303f5932516.zip#egg=bleurt

     | 16.5 MB 105.0 MB/s 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
  Created wheel for bleurt: filename=BLEURT-0.0.2-py3-none-any.whl size=16454005 sha256=b5870920f8319b4d09aca9ad4e89ceda8553a82a3192bccc3d747357bdc27a08
  Stored in directory: /tmp/pip-ephem-wheel-cache-_sv75efp/wheels/d6/8b/6d/18f51ff9a19a24c6f73dd939e4ce5cd80972275d1ec0c25dc0
Successfully built bleurt
  Attempting uninstall: fsspec
    Found existing install

In [ ]:
from os import path
from google.colab import drive
import json

CACHE_PATH = '/content/drive/MyDrive/followup/cache.json'
drive.mount('/content/drive', force_remount=True)

if not path.exists(CACHE_PATH):
  print("Cache file not found")
  cache = {}
else:
  with open(CACHE_PATH) as f:
    cache = json.load(f)

Mounted at /content/drive


In [ ]:
import sacrebleu
from bleurt import score

checkpoint = "BLEURT-20-D6"
scorer = score.BleurtScorer(checkpoint)

def bleu(candidate: str, references: list[str]) -> float:
  return sacrebleu.sentence_bleu(candidate, references).score / 100

def bleurt(candidate: str, references: list[str]) -> float:
  scores = scorer.score(references=references,
                        candidates=[candidate] * len(references))
  assert isinstance(scores, list) and len(scores) == len(references)
  return max(scores)

In [ ]:
import itertools
import datasets
import sacrebleu

dataset = datasets.load_dataset("ambig_qa", split="train")

for example in itertools.islice(dataset, 30):
  question = example['question']
  questions = list(set([q for qa in example["annotations"]["qaPairs"] for q in qa["question"]]))
  if len(questions) > 1:
    print(question, questions)

When did the simpsons first air on television? ['When did the Simpsons first air on television as an animated short on the Tracey Ullman Show?', 'When did the Simpsons first air as a half-hour prime time show?']
What is the legal age of marriage in usa? ['What is the legal age of marriage, without parental consent or other authorization, in Nebraska?', 'What is youngest legal age of marriage possible in some US states when circumstances permit?', 'What is the legal age of marriage, without parental consent or other authorization, in all but two states in the usa?', 'What is the legal age of marriage, without parental consent or other authorization, in Mississippi?']
Who starred in barefoot in the park on broadway? ['Who starred in barefoot in the park on broadway as Mrs. Banks?', 'Who starred in barefoot in the park on broadway as Telephone Man?', 'Who starred in barefoot in the park on broadway as Victor Velasco?', 'Who starred in barefoot in the park on broadway as Delivery Man?', 'W

In [ ]:
import collections
import datasets
import tqdm
import openai
from google.colab import userdata


SEPARATOR = "\n * "

client = openai.OpenAI(api_key=userdata.get('openaiApiKey'))


def parse_json_answer(response: str):
  return json.loads(
      response.choices[0].message.content.split('```json')[1].split('```')[0])

def get_model_response(id, prompt):
  if id in cache:
    return cache[id]
  response = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt}],
    model="gpt-4o-mini",
  )
  cache[id] = parse_json_answer(response)
  return cache[id]


dataset = datasets.load_dataset("ambig_qa", split="train")
filtered_examples = []
cnt = 0
bleu_metrics = collections.defaultdict(list)
bleurt_metrics = collections.defaultdict(list)

_MODELS = ("gpt-4o-mini",)
_CLARIFICATION_KEY = "clarification"

for example in tqdm.tqdm(dataset):
  question = example['question']
  questions = list(set([q for qa in example["annotations"]["qaPairs"] for q in qa["question"]]))
  prompt = f"""\
The question "{question}" can be interpreted in multiple ways, for example any of the following interpretations are valid:
 * {SEPARATOR.join(questions)}

Please write a concise follow-up clarification question that you could ask in order to distinguish between the possible interpretations, mentioning at least two of the options or pointing to the missing necessary context.
For example, the question may start with "Did you mean A or B", or "Are you referring to which of", "Can you clarify if", etc.
Answer as a JSON list of 4 to 8 strings, each string being a single possible follow-up clarification question."""

  answer_prompt = f"""\
Please answer the following question without making any assumption of my intent and only if \
you are sure there is a unique answer. \
You may reason step by step and explain your thinking but finish with json block \
with a key "answer" using a short concise phrase if you know the answer. \
If you don't know the answer or there are multiple possible answers due to ambiguity,
use the key "{_CLARIFICATION_KEY}" with a clarification questions in order to find out what I meant.

For example, if the question is "Who won the last soccer world cup?" you may reply:
```json
{{
    "{_CLARIFICATION_KEY}": "Do you mean the men or the women's world cup?"
}}
```

Question: {question}"""
  if len(questions) > 1:
    follow_ups = get_model_response(example['id'], prompt)
    example["prompt"] = prompt
    example["follow_ups"] = follow_ups
    for model in _MODELS:
      answer_key = f"answer-{model}"
      example[answer_key] = get_model_response(f"{answer_key}-{example['id']}", answer_prompt)
      if _CLARIFICATION_KEY in example[answer_key]:
        bleu_metrics[model].append(bleu(example[answer_key][_CLARIFICATION_KEY], follow_ups))
        bleurt_metrics[model].append(bleurt(example[answer_key][_CLARIFICATION_KEY], follow_ups))
      else:
        bleu_metrics[model].append(0)
        bleurt_metrics[model].append(0)

    filtered_examples.append(example)
    cnt += 1
    if cnt == 500:
      break

len(filtered_examples)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/23.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/7.20M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10036 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2002 [00:00<?, ? examples/s]

 10%|█         | 1012/10036 [13:57<2:04:28,  1.21it/s]


500

In [ ]:
# Save Cache to Drive

with open(CACHE_PATH, 'w') as f:
  json.dump(cache, f)

In [ ]:
import re

def normalize(answer:str) -> str:
  return re.sub('\W', '', answer.lower())


answer_correct = []

for ex in filtered_examples:
  answers = [normalize(answer) for pair in ex['annotations']['qaPairs'] for answers in pair['answer'] for answer in answers]
  if 'answer' in ex['answer-gpt-4o-mini']:
    answer_correct.append(any(normalize(ex['answer-gpt-4o-mini']['answer']) in answer for answer in answers))

np.mean(answer_correct)

0.5571030640668524

In [ ]:
for ex in filtered_examples[:50]:
  print(ex['question'], ex['answer-gpt-4o-mini'], ex['follow_ups'])

When did the simpsons first air on television? {'answer': 'December 17, 1989'} ['Did you mean when the Simpsons first aired as a half-hour prime time show or when they first appeared as shorts on the Tracey Ullman Show?', 'Are you referring to the premiere of the full television series or the initial animated shorts?', "Can you clarify if you're asking about the debut on the Tracey Ullman Show or the start of the weekly series?", 'Are you inquiring about the first airing as an animated short or the first full-length episode?', 'Could you specify if you mean the original short format or the later prime time episodes?', 'Do you want to know about the first airing of the series or the initial shorts that introduced the characters?', 'Is your question focused on the start of the series or its origins on the Tracey Ullman Show?']
What is the legal age of marriage in usa? {'clarification': 'Are you asking about a specific state or looking for a general answer for the entire country?'} ['Did 

In [ ]:
import numpy as np

def print_metrics(metrics):
  metric_array = np.array(metrics)
  asked_follow_up = metric_array > 0
  return np.mean(asked_follow_up), np.mean(metric_array[asked_follow_up])

print('bleu', print_metrics(bleu_metrics[_MODELS[0]]))
print('bleurt', print_metrics(bleurt_metrics[_MODELS[0]]))

bleu (0.282, 0.24456165286407203)
bleurt (0.282, 0.502644677956899)


# Ejemplos

What is the legal age of marriage in usa?
> Are you asking about a specific state or looking for a general answer for the entire country?
> * Did you mean the legal age of marriage in a specific state, such as Mississippi, or in general across the USA?
> * Are you asking about the legal age of marriage without parental consent, or are you interested in the minimum age considering exceptions in some states?
> * Can you clarify if you are referring to the legal age of marriage in Nebraska, or are you interested in the youngest legal age of marriage possible in any state?
> * Are you looking for the legal age of marriage across all states, or just in a particular state like Nebraska or Mississippi?

When was the last time uga won a national championship?
> Are you asking about football or another sport?
> * Can you clarify if you're referring to the football national championship or perhaps another sport like gymnastics or baseball?
> * Are you asking about the most recent national championship in football, equestrian, or another sport?
> * Did you mean the last national championship won in football, or are you interested in sports like tennis or swimming?
> * Can you specify if you are inquiring about football or one of the other sports, such as track and field or golf?
> * Are you referring to the last national championship in football, or are you interested in achievements in equestrian or gymnastics?
> * Can you clarify if you're asking about a specific sport like baseball or swimming, or should I focus on football?
> * Did you mean the most recent national championship for football, or are you curious about something like tennis or diving?

Voice of the snake in the jungle book?
> Which adaptation of The Jungle Book are you referring to: the animated version or the live-action version?
> * Did you mean the voice of Kaa in the 1967 film or the 2016 film?
> * Are you asking about the voice of Kaa in the animated movie or the live-action adaptation?
> * Can you clarify if you're interested in the original film or a TV series version?
> * Are you referring to the character Kaa in the 1967 film, the 2016 film, or another adaptation?
> * Did you mean to ask about Kaa's voice in the animated film or the television series?
> * Can you specify if you are inquiring about the classic film or a modern adaptation?

How many jury members in a criminal trial?
> Which country or legal system are you referring to?
> * Did you mean the number of jury members for serious felonies in the US or for a criminal trial in Florida?
> * Are you referring to the jury size in a criminal trial in Scotland or in the Republic of Ireland?
> * Can you clarify if you are asking about WWII England and Wales or modern-day criminal trials in Florida?
> * Do you want to know the minimum jury size in a Florida trial or in a serious felony trial in the US?
> * Are you interested in the number of jurors in a criminal trial for murder in the UK or general criminal trials in the US?
> * Can you specify if you mean jury members in a modern trial in Florida or the historical context of WWII England?
> * Is your question about jury members in a criminal trial in the Republic of Ireland or in Scotland?

Who has conceded the most goals in the premiership?
> Are you asking for the record for a specific season or overall history of the Premier League?
> * Can you clarify if you are referring to a specific player or team?
> * Did you mean which goalkeeper or which team has conceded the most goals?
> * Are you asking about individual players, like a goalkeeper, or overall team statistics?
> * Could you specify if you're interested in the current season or historical data?
> * Are you asking about which team has allowed the most goals overall or in a specific season?
> * Is your question about a particular timeframe, like all-time records or just the current season?

What day is the new star wars movie?
> Which specific Star Wars movie are you asking about?
> * Are you asking about the release date of the movie from 2015 or 2017?
> * Can you clarify if you're referring to the movie that released in 2005 or 2015?
> * Did you mean the Star Wars movie that came out in 2015 or the one from 2017?
> * Are you looking for the release date of the 2017 film or the 2005 film?
> * Can you specify if you're asking about the 2015 or the 2017 Star Wars movie?
> * Are you referring to the release date in 2005 or 2017?
> * Could you clarify if you mean the new movie from 2015 or another one?